## Cliente ejemplo para analizar el consumo de CO2

Importamos las librerías que vamos a utilizar

In [32]:
# Algunas de las de siempre
import numpy as np
import pandas as pd

# Las que vamos a necesitar para hacer los llamados y para lidiar con los datos en formato json
import requests
from flask import  Flask, request, jsonify, render_template
import json



***

## **ENDPOINT PARA ENTRENAR:** entrenar el modelo

In [43]:
df = pd.read_csv('Data/emision-co2-autos_limpio.csv')
df.dropna(inplace=True)

In [44]:
# Definimos los datos que vamos a pasar en el request
base=df.to_json() #dataframe en formato json
lista_predictores=df.drop(columns=['vehiculo_marca', 'vehiculo_modelo', 'vehiculo_tipo',
                                   'vehiculo_traccion','vehiculo_potencia','lca_numero', 
                                   'vehiculo_id_motor','vehiculo_id_motor','fecha_firma',
                                   'ensayo_gei_numero','ensayo_gei_laboratorio','id_etiqueta',
                                  'vehiculo_tipo_transmision','vehiculo_tipo_combustible', 
                                   'vehiculo_standard_emision', 'emision_CO2']).columns.values #features para entrenar el modelo
variable_target='emision_CO2' 
directory='/media/ignacio/DH/desafio-final' #dirección donde vamos a guardar el modelo  
name='predecir_consumo_Co2' #nombre modelo

url = 'http://localhost:5000/entrenar_modelo' #definimos la URL para hacer el llamado local con el endpoint correspondiente

In [46]:
# Hacemos el pedido y vemos los resultados.
modelo = requests.post(url,
                       json = {'base' : base,
                               'lista_predictores':list(lista_predictores),
                               'target':variable_target,
                               'directory':directory,
                               'name':name})
display(modelo.json())

{'r2_resultado_test': 0.8430373731777071,
 'r2_resultado_train': 0.892148002360325}

***

## **SEGUNDA FUNCIÓN/ENDOPOINT:** coeficientes del modelo

In [6]:
#Definimos los datos que vamos a pasar en el pedido
dir_modelo='/media/ignacio/DH/modelos' #dirección donde se encuentra el modelo entrenado
nombre_modelo='predictor_jugadores' #nombre del modelo entrenado
palette='hot' #paleta de colores para usar en el gráfico de barras
size=(25,8) #tamaño de la figura
predictores=list(lista_predictores) #lista con los features en el mismo orden en el que fueron pasados en el modelo
url = 'http://localhost:5000/plot_coeficientes' #definimos la URL para hacer el llamado local con el endpoint correspondiente

In [7]:
# Hacemos el pedido e imprimimos el output
coef = requests.post(url,
                     json = {'direccion' : dir_modelo,
                             'nombre_modelo':nombre_modelo,
                             'paleta_colores':palette,
                             'tamano':size,
                             'predictores':predictores})
print(coef.json())

{"coeficientes":{"potential":4.0398941403,"age":3.4317419111,"f\u00edsico":1.0093486869,"amagues":0.8443859787,"defensa":0.6407756637,"ritmo":0.5069634046,"disparos":0.322959432,"pases":0.0728771556,"weight_kg":0.0310999457,"height_cm":0.0164526487}}


***

## **TERCERA FUNCIÓN/ENDOPOINT:** predicciones

In [8]:
#Definimos las variables que vamos a utilizar para el pedido
direccion_modelo='/media/ignacio/DH/modelos' #dirección donde se encuentra el modelo grabado
nombre_modelo='predictor_jugadores' #nombre del modelo grabado
nombre_escalador='predictor_jugadores_scaler' #nombre del escalador que entrenamos cuando corrimos el modelo
features=df.drop(columns=['ID','full_name','club','puntaje_global']).iloc[3].to_json() #elegimos features de un jugador 
#cualquiera de la base para chequear que funcione bien el llamado
url = 'http://localhost:5000/prediccion' #definimos la URL para hacer el llamado local con el endpoint correspondiente


In [9]:
#Hacemos el llamado
pred = requests.get(url, 
                    params = {'direccion':direccion_modelo,
                              'nombre_modelo':nombre_modelo,
                              'nombre_scalador':nombre_escalador,
                              'features':[features]})
print(pred.json())

{'prediccion': 83.63296997609625}


### ¡Ahí tenemos nuestra predicción! ¡Congrats!
<img src="img/we_did_it.jpg" alt="Drawing" style="width: 300px;"/>